## dataset

In [31]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [18]:
import os
working_dir = os.getcwd()

In [19]:
from transformers import TextDataset
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#dataset_train = TextDataset(tokenizer=tokenizer, 
                                #file_path="/notebook/greenAI/wikitext-103/wiki.train.tokens", 
                                #block_size=512)

dataset_test = TextDataset(tokenizer=tokenizer, 
                                file_path="/notebook/greenAI/wikitext-103/wiki.valid.tokens", 
                                block_size=1024)

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu") 

/opt/.pyenv/versions/3.8.10/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


In [20]:
seq_len = 1024

In [21]:
def collate_fn(item):
        """Collate function for DataLoader
        Args:
            item (List[dict[str, List[int]]])
        Returns:
            (dict[str, torch.Tensor]):
        """
        keys = item[0].keys()
        dic = {
            key: torch.tensor([x[key] for x in item])
            for key in keys
        }
        return dic
    
def __len__(self):
        return len(self.filelist)

In [22]:
dataset_test[:15]

tensor([[ 220,  198,  796,  ...,  262, 2619, 6896],
        [ 764,  383, 7840,  ...,  290,  262, 1279],
        [2954,   29,  423,  ...,  764,  679,  373],
        ...,
        [ 358, 1812, 3845,  ...,  837,  351, 6135],
        [2488,   13,   31,  ..., 2700,  373,  691],
        [1315, 2488,   11,  ..., 8069,  290,  468]])

In [23]:
val_dataloader = torch.utils.data.DataLoader(dataset=dataset_test[:15], 
                                             batch_size=5, 
                                             drop_last=True)

In [24]:
import torch
class Object(object):
    pass

args = Object()
args.per_gpu_train_batch_size = 8
args.per_gpu_eval_batch_size = 2
args.n_gpu = 1
args.num_train_epochs = 25
args.seed = 42
args.mlm = False
args.device = device
args.output_dir = working_dir 
args.fp16 = False
args.max_grad_norm = 1.0
args.logging_steps = 500.0
args.eval_batch_size = 6
args.save_total_limit = 2
args.is_factorized = True
args.local_rank = -1
args.max_steps = -1
args.per_gpu_train_batch_size = 8
args.per_gpu_eval_batch_size = 2


In [9]:
!pip3 install path

You should consider upgrading via the '/opt/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


## regular

In [34]:
import sys
import os

current = globals()['_dh'][0]
 
# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(current)
 
# adding the parent directory to
# the sys.path.
sys.path.append(parent)
from help_trainer import evaluate


In [35]:
from transformers import GPT2Model, GPT2Config, GPT2LMHeadModel
from src.classes.gpt_med_config import GPT2MedConfig
# Initializing a GPT2 configuration
configuration = GPT2MedConfig()



In [141]:
# Initializing a model from the configuration
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

#evaluate(args, model.to(device), dataset_test, tokenizer)

In [49]:
!pip3 install tqdm

You should consider upgrading via the '/opt/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


In [38]:
for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
    
    eval_loss = eval_loss / nb_eval_steps
    #perplexity = perplexity / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [39]:
eval_loss = 0.0
perplexity = 0.0
nb_eval_steps = 0

with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        print (len(outputs[0]), len(outputs[1]))
        lm_loss = outputs[0]
        eval_loss += lm_loss.mean().item()
        perplexity += torch.exp(torch.tensor(eval_loss))
        nb_eval_steps += 1
    model.train()
    eval_loss = eval_loss / nb_eval_steps
    #perplexity = perplexity / nb_eval_steps
    print ("loss", eval_loss)
    perplexity = torch.exp(torch.tensor(eval_loss))
print (perplexity)

NameError: name 'tqdm' is not defined

## rank 72

In [146]:
from src.classes.gpt2_tt import GPT2_TT_Model
configuration = GPT2MedConfig()

import torch.distributed as dist
   
model = GPT2_TT_Model(configuration, rank = 72)
model.to(device)

GPT2_TT_Model(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): FactorizationTTMLinear(
            (ttm): TTM(
              (tt): PlainTTMContainer(
                (cores): ParameterList(
                    (0): Parameter containing: [torch.cuda.FloatTensor of size 1x16x16x72 (GPU 2)]
                    (1): Parameter containing: [torch.cuda.FloatTensor of size 72x16x16x72 (GPU 2)]
                    (2): Parameter containing: [torch.cuda.FloatTensor of size 72x4x16x1 (GPU 2

In [41]:
print(sum(p.numel() for p in model.parameters()))

218303488


In [42]:
import torch
model.load_state_dict(torch.load('/notebook/greenAI/out_transformer_0_v4/checkpoint-6000/model_tt.pth', map_location= torch.device(device)))


evaluate(args, model.to(device), dataset_test, tokenizer)

Evaluating: 100% 243/243 [00:17<00:00, 13.89it/s]


{'perplexity': tensor(38.4058), 'loss': 3.6482085612575705}

In [43]:
from tqdm import tqdm

model.to(device).eval()
i = 0
with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        #print("INPUT", tokenizer.decode(input_ids[0]), "\n")
        #print("REAL", tokenizer.decode(label_ids[0]), "\n")
        print ("\n")
        print("input_ids[0][-10:]", tokenizer.decode(input_ids[0][-20:]))
        print ("\n")
        print("label_ids[0][-11:]", tokenizer.decode(input_ids[0][-21:]))
        print ("\n")
        print("output", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-20:]))
        #print("GENERATED", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
        print("\n\n\n")
        i += 1
        if i > 10: break

0it [00:00, ?it/s]

1it [00:00,  2.73it/s]

input_ids[0][-10:]  section east of Crete, and along only the north @-@ west coast of the Black Sea


label_ids[0][-11:]  the section east of Crete, and along only the north @-@ west coast of the Black Sea


output  of of thete. and is the the eastern coast-@ eastern coast of the Black Sea.








2it [00:00,  2.73it/s]

input_ids[0][-10:]  Hesperia to New Era, bypassing Ferry. A larger change around 1969 reconfigured the highway design


label_ids[0][-11:]  from Hesperia to New Era, bypassing Ferry. A larger change around 1969 reconfigured the highway design


output peria to Howarday. replacinging the,  new section was the wasfigured the highway to to








3it [00:01,  2.72it/s]

input_ids[0][-10:] 
 = = = Historic districts = = = 
 
 Meridian has nine historic districts that are


label_ids[0][-11:]  
 = = = Historic districts = = = 
 
 Meridian has nine historic districts that are


output  The = = Hot buildings = =  FactoryReloaded @#& The Main been historic districts, are part






## Bookcorpus

In [124]:
from datasets import load_dataset
test_ds = load_dataset("bookcorpus",split='train[:478]')

Found cached dataset bookcorpus (/root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f)


In [125]:
len(test_ds)

478

In [126]:
test_ds[:10]

{'text': ['usually , he would be tearing around the living room , playing with his toys .',
  'but just one look at a minion sent him practically catatonic .',
  "that had been megan 's plan when she got him dressed earlier .",
  "he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .",
  'she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .',
  "`` are n't you being a good boy ? ''",
  'she said .',
  'mason barely acknowledged her .',
  'instead , his baby blues remained focused on the television .',
  'since the movie was almost over , megan knew she better slip into the bedroom and finish getting ready .']}

In [127]:
encoded_dataset = test_ds.map(lambda examples: tokenizer(examples['text'], max_length=1025, truncation=True), batched=True)
len(encoded_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

478

In [128]:
encoded_dataset[0]

{'text': 'usually , he would be tearing around the living room , playing with his toys .',
 'input_ids': [23073,
  837,
  339,
  561,
  307,
  24447,
  1088,
  262,
  2877,
  2119,
  837,
  2712,
  351,
  465,
  14958,
  764],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [129]:
import torch

class BookCorpusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        assert idx < len(self.encodings)
        item = {}
        item['decoder_attention_mask'] = self.encodings[idx]['attention_mask']
        item['input_ids'] =  self.encodings[idx]['input_ids']
        return item       

    def __len__(self):
        return len(self.encodings)

val_dataset = BookCorpusDataset(encoded_dataset)

In [130]:
def collate_fn(item):
    """Collate function for DataLoader
    Args:
        item (List[dict[str, List[int]]])
    Returns:
        (dict[str, torch.Tensor]):
    """
    keys = item[0].keys()
    dic = {
        key: torch.tensor([x[key] for x in item])
        for key in keys
    }
    return dic

In [131]:
val_dataloader = DataLoader(
    val_dataset,
    collate_fn = collate_fn,
    batch_size=1,
    shuffle=False,
)

In [132]:
len(val_dataloader)

478

In [133]:
i = 0
for batch_idx, sample in tqdm(enumerate(val_dataloader)):
    #print(batch_idx, sample)
    input_ids, label_ids = sample['input_ids'][0][:seq_len - 1], sample['input_ids'][0][1:seq_len]
    #input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)
    # average(on number of tokens) negative log-likelihood for each token is returned as the loss
    outputs = model(input_ids)
    #print("INPUT", tokenizer.decode(input_ids[0]), "\n")
    #print("REAL", tokenizer.decode(label_ids[0]), "\n")
    print ("\n")
    print("input_ids[0][-20:]", tokenizer.decode(input_ids[-10:]))
    print ("\n")
    print("label_ids[0][-21:]", tokenizer.decode(input_ids[-11:]))
    print ("\n")
    print("output", tokenizer.decode(outputs.logits.argmax(dim=-1)[-10:]))
    #print("GENERATED", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
    print("\n\n\n")
    i += 1
    if i > 10: break
    print ("\n\n")

4it [00:00, 15.15it/s]



input_ids[0][-20:]  around the living room, playing with his toys.


label_ids[0][-21:]  tearing around the living room, playing with his toys.


output  in edges room, and with his hands,.









input_ids[0][-20:]  at a minion sent him practically catatonic.


label_ids[0][-21:]  look at a minion sent him practically catatonic.


output  the few. to to intoatonic..









input_ids[0][-20:] 's plan when she got him dressed earlier.


label_ids[0][-21:] gan's plan when she got him dressed earlier.


output em. for I was the. up this.









input_ids[0][-20:] ason was often exposed to things that were older.


label_ids[0][-21:]  mason was often exposed to things that were older.


output  and a seen to the that were not than He









8it [00:00, 15.85it/s]



input_ids[0][-20:]  a such a good talker for his age.


label_ids[0][-21:]  was a such a good talker for his age.


output  good a good kider. the age..









input_ids[0][-20:]  aren't you being a good boy? ''


label_ids[0][-21:] `` aren't you being a good boy? ''


output  theameless the? a little person?""









input_ids[0][-20:] she said.


label_ids[0][-21:] she said.


output  was..









input_ids[0][-20:] mason barely acknowledged her.


label_ids[0][-21:] mason barely acknowledged her.


output ., survived the own.









10it [00:00, 14.23it/s]



input_ids[0][-20:] , his baby blues remained focused on the television.


label_ids[0][-21:] instead, his baby blues remained focused on the television.


output  the wife was, the on the music show He









input_ids[0][-20:]  better slip into the bedroom and finish getting ready.


label_ids[0][-21:]  she better slip into the bedroom and finish getting ready.


output  not into the role and get the ready for I









input_ids[0][-20:]  was grateful that he looked nothing like his father.


label_ids[0][-21:]  she was grateful that he looked nothing like his father.


output  still for she was at like her...






In [134]:
i = 0
for batch_idx, sample in tqdm(enumerate(val_dataloader)):
    #print(batch_idx, sample)
    input_ids, label_ids = sample['input_ids'][0][:seq_len - 1], sample['input_ids'][0][1:seq_len]
    #input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)
    # average(on number of tokens) negative log-likelihood for each token is returned as the loss
    outputs = model(input_ids)
    #print("INPUT", tokenizer.decode(input_ids[0]), "\n")
    #print("REAL", tokenizer.decode(label_ids[0]), "\n")
    print ("\n")
    print("input_ids[0][-20:]", tokenizer.decode(input_ids[-10:]))
    print ("\n")
    print("label_ids[0][-21:]", tokenizer.decode(input_ids[-11:]))
    print ("\n")
    print("output", tokenizer.decode(outputs.logits.argmax(dim=-1)[-10:]))
    #print("GENERATED", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
    print("\n\n\n")
    i += 1
    if i > 10: break
    print ("\n\n")

4it [00:00, 15.74it/s]



input_ids[0][-20:]  around the living room, playing with his toys.


label_ids[0][-21:]  tearing around the living room, playing with his toys.


output  in edges room, and with his hands,.









input_ids[0][-20:]  at a minion sent him practically catatonic.


label_ids[0][-21:]  look at a minion sent him practically catatonic.


output  the few. to to intoatonic..









input_ids[0][-20:] 's plan when she got him dressed earlier.


label_ids[0][-21:] gan's plan when she got him dressed earlier.


output em. for I was the. up this.









input_ids[0][-20:] ason was often exposed to things that were older.


label_ids[0][-21:]  mason was often exposed to things that were older.


output  and a seen to the that were not than He









8it [00:00, 15.98it/s]



input_ids[0][-20:]  a such a good talker for his age.


label_ids[0][-21:]  was a such a good talker for his age.


output  good a good kider. the age..









input_ids[0][-20:]  aren't you being a good boy? ''


label_ids[0][-21:] `` aren't you being a good boy? ''


output  theameless the? a little person?""









input_ids[0][-20:] she said.


label_ids[0][-21:] she said.


output  was..









input_ids[0][-20:] mason barely acknowledged her.


label_ids[0][-21:] mason barely acknowledged her.


output ., survived the own.









10it [00:00, 14.45it/s]



input_ids[0][-20:] , his baby blues remained focused on the television.


label_ids[0][-21:] instead, his baby blues remained focused on the television.


output  the wife was, the on the music show He









input_ids[0][-20:]  better slip into the bedroom and finish getting ready.


label_ids[0][-21:]  she better slip into the bedroom and finish getting ready.


output  not into the role and get the ready for I









input_ids[0][-20:]  was grateful that he looked nothing like his father.


label_ids[0][-21:]  she was grateful that he looked nothing like his father.


output  still for she was at like her...






In [135]:
args.eval_batch_size = 1

In [136]:
len(val_dataset)

478

In [137]:
eval_dataloader = DataLoader(val_dataset, batch_size=1)
len(eval_dataloader)

478

In [143]:
eval_loss = 0.0
perplexity = 0.0
nb_eval_steps = 1
model.eval().to(device)
i = 0
losses = []
for idx, sample in tqdm(enumerate(val_dataloader), desc="Evaluating"):
    input_ids = sample['input_ids'][0][:seq_len]
    label_ids = sample['input_ids'][0][1:len(input_ids)]
    input_ids = input_ids[:len(input_ids) - 1]
    #input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)

    with torch.no_grad():
        outputs = model(input_ids, labels=label_ids)
        if i < 10:
            print ("\n")
            print("input_ids[0][-20:]", tokenizer.decode(input_ids[-10:]))
            print ("\n")
            print("label_ids[0][-21:]", tokenizer.decode(input_ids[-11:]))
            print ("\n")
            print("output", tokenizer.decode(outputs.logits.argmax(dim=-1)[-10:]))
            #print("GENERATED", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
            print("\n\n\n")
            i += 1
        
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
            perplexity += torch.exp(torch.tensor(eval_loss))
            losses.append(eval_loss)
            nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
#perplexity = perplexity / nb_eval_steps
perplexity = torch.exp(torch.tensor(eval_loss))

Evaluating: 8it [00:00, 35.32it/s]



input_ids[0][-20:]  tearing around the living room, playing with his toys


label_ids[0][-21:]  be tearing around the living room, playing with his toys


output  up the world room, and with his toys,






input_ids[0][-20:]  look at a minion sent him practically catatonic


label_ids[0][-21:]  one look at a minion sent him practically catatonic


output  at the picture's to to toatonic.






input_ids[0][-20:] gan's plan when she got him dressed earlier


label_ids[0][-21:]  megan's plan when she got him dressed earlier


output icn first to she was her. up.






input_ids[0][-20:]  mason was often exposed to things that were older


label_ids[0][-21:] , mason was often exposed to things that were older


output ama, a seen to the that were not than






input_ids[0][-20:]  was a such a good talker for his age


label_ids[0][-21:]  he was a such a good talker for his age


output  so good a good studenter. her age.






input_ids[0][-20:] `` aren't you being a good boy?


label

Evaluating: 16it [00:00, 36.48it/s]



input_ids[0][-20:] instead, his baby blues remained focused on the television


label_ids[0][-21:] instead, his baby blues remained focused on the television


output  of and name brother,. on the music screen






input_ids[0][-20:]  she better slip into the bedroom and finish getting ready


label_ids[0][-21:]  knew she better slip into the bedroom and finish getting ready


output  had not into a bathroom and get the ready for






Evaluating: 478it [00:16, 29.32it/s]


In [144]:
perplexity

tensor(5649.4297)

In [145]:
eval_loss

8.639309839768844

In [148]:
eval_loss = 0.0
perplexity = 0.0
nb_eval_steps = 1
model.eval().to(device)
i = 0
losses = []
for idx, sample in tqdm(enumerate(val_dataloader), desc="Evaluating"):
    input_ids = sample['input_ids'][0][:seq_len]
    label_ids = sample['input_ids'][0][1:len(input_ids)]
    input_ids = input_ids[:len(input_ids) - 1]
    #input_ids, label_ids = sample[:seq_len], sample[1:seq_len+1]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)

    with torch.no_grad():
        outputs = model(input_ids, labels=label_ids)
        if i < 10:
            print ("\n")
            print("input_ids[0][-20:]", tokenizer.decode(input_ids[-10:]))
            print ("\n")
            print("label_ids[0][-21:]", tokenizer.decode(input_ids[-11:]))
            print ("\n")
            print("output", tokenizer.decode(outputs.logits.argmax(dim=-1)[-10:]))
            #print("GENERATED", tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
            print("\n\n\n")
            i += 1
        
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
            perplexity += torch.exp(torch.tensor(eval_loss))
            losses.append(eval_loss)
            nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
#perplexity = perplexity / nb_eval_steps
perplexity = torch.exp(torch.tensor(eval_loss))

Evaluating: 2it [00:00, 12.56it/s]



input_ids[0][-20:]  tearing around the living room, playing with his toys


label_ids[0][-21:]  be tearing around the living room, playing with his toys


output  MoonlightDustgi unders NortcanoDust Jeffreyodecano






input_ids[0][-20:]  look at a minion sent him practically catatonic


label_ids[0][-21:]  one look at a minion sent him practically catatonic


output  stigma stigma stigma utilities servicing stigma stigma stigma stigma Optional






input_ids[0][-20:] gan's plan when she got him dressed earlier


label_ids[0][-21:]  megan's plan when she got him dressed earlier


output  servicing InvestigativeUSA stigma stigma stigma stigma stigma UNCLASSIFIED stigma






Evaluating: 6it [00:00, 13.57it/s]



input_ids[0][-20:]  mason was often exposed to things that were older


label_ids[0][-21:] , mason was often exposed to things that were older


output phyphyphycosphyphyphyphyphyphy






input_ids[0][-20:]  was a such a good talker for his age


label_ids[0][-21:]  he was a such a good talker for his age


output א stigma OptionalGameplay stigmaownersGameplay fountainowners stigma






input_ids[0][-20:] `` aren't you being a good boy?


label_ids[0][-21:] `` aren't you being a good boy?


output  premium Soros authorize Me reflective Thursday interpol interpol appreciate interpol






Evaluating: 8it [00:00, 14.10it/s]



input_ids[0][-20:] she said


label_ids[0][-21:] she said


output  avenues X






input_ids[0][-20:] mason barely acknowledged her


label_ids[0][-21:] mason barely acknowledged her


output  periodic25'd servicing servicing






input_ids[0][-20:] instead, his baby blues remained focused on the television


label_ids[0][-21:] instead, his baby blues remained focused on the television


output  ha isolatedbinding Liver Apprentice unders shades shades unders vitamins






Evaluating: 12it [00:00, 14.41it/s]



input_ids[0][-20:]  she better slip into the bedroom and finish getting ready


label_ids[0][-21:]  knew she better slip into the bedroom and finish getting ready


output ************ appreciate************ appreciate appreciate appreciate appreciate appreciate appreciatelos






Evaluating: 478it [00:30, 15.47it/s]


In [149]:
perplexity

tensor(24157.7266)

In [150]:
eval_loss

10.092359629544346